In [1]:
using Distributed
addprocs(4)
nprocs()

5

In [2]:
@everywhere import Pkg; 
@everywhere Pkg.activate("..")
@everywhere using DigitalMusicology
@everywhere using Interact

@everywhere include("../src/Polygrams.jl");

Activating environment at `~/DCML/schemata_code/Project.toml`
      From worker 5:	Activating environment at `~/DCML/schemata_code/Project.toml`
      From worker 4:	Activating environment at `~/DCML/schemata_code/Project.toml`
      From worker 2:	Activating environment at `~/DCML/schemata_code/Project.toml`
      From worker 3:	Activating environment at `~/DCML/schemata_code/Project.toml`


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-14021631262568913254\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-16120713156675459959\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        require([\"/assetserver/905b1a7a2a6966b0e3b9a1f3eda6f274db443f93-dist/main.js\"], function (webIOModule) {\n            webIOModule.load_ipython_extension();\n        });\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<div\n    id=\"webio-warning-14021631262568913254\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    <!-- TODO: link to installation docs. -->\n</div>\n")

[ Info: waiting for lock on pidfile at "/home/deguernel/.jlassetregistry.lock"
[ Info: waiting for lock on pidfile at "/home/deguernel/.jlassetregistry.lock"


In [3]:
@everywhere corpusdir = joinpath("..", "..", "mozart_sonatas")
@everywhere lexicon = Polygrams.loadlexicon(joinpath("..", "data", "lexicon.json"));
@everywhere DigitalMusicology.usedir(joinpath(corpusdir, "musicxml"));
@everywhere annotdir = joinpath(corpusdir, "groups");
@everywhere DigitalMusicology.allpieces()

In [4]:
#@everywhere corpus = ["K310-2"]; 
@everywhere schemata_list = ["folia.3", "folia.4"]; 

@everywhere corpus = DigitalMusicology.allpieces();
#@everywhere schemata_list = keys(lexicon);



Leave this code unchanged (but run the cells):

In [5]:
@distributed for piece in corpus
    for schema in schemata_list
        
        infile = getpiece(piece, :file, :musicxml);
        outfile = Polygrams.annotfilename(piece, schema, annotdir)
        if !isfile(outfile) || mtime(infile) > mtime(outfile)
        
            notes = getpiece(piece, :notes, :musicxml, type=:notes, keepids=true);
            xml = getpiece(piece, :xml, :musicxml, keepids=true)
            timesigs = getpiece(piece, :timesigs, :musicxml)[1]
            timesig = content(timesigs[1]);
            barlen = duration(timesig)
            beatfactor = denominator(timesig)
            pattern = lexicon[schema];
            featurefs = [
                poly -> beatfactor * Polygrams.totalduration(poly), # duration
                Polygrams.voicedist,                                # voice dist
                poly -> beatfactor * Polygrams.instageskip(poly)    # stage skip
                #, poly -> Polygrams.polymweight(poly, timesigs) # metric weight
            ]
            featureweights = [1, -1, -2]; # maaaaaagic
        
            polyitr = Polygrams.schemamatches(notes, [pattern], barlen, barlen);
            polys = collect(polyitr);
            sorted = Polygrams.sortbyheuristics(polys, featurefs, featureweights);
            print(length(sorted))
            #scores = Dict(poly => Polygrams.rate(poly, featurefs, featureweights) for poly in polys);
        
            best = Polygrams.bestmatchestime(sorted)
            groups = map(b -> Polygrams.findcompetitors(Polygrams.polyssharetime, b, sorted), best)
            groups = map(g -> g[1:min(25, length(g))], groups)
            sort!(groups, by=g -> onset(g[1][1][1]))
            Polygrams.savegroups(piece, schema, groups, annotdir)
            println("saved!")
        else
            println("Already up to date.")
        end
    end
end

Task (runnable) @0x00007f425d601ae0

      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worker 2:	Already up to date.
      From worke

┌ Warning: unfolding for time signatures is not implemented yet!
└ @ DigitalMusicology.Corpora.Directory ~/.julia/packages/DigitalMusicology/6r3xS/src/corpora/Directory.jl:67
┌ Warning: unfolding for time signatures is not implemented yet!
└ @ DigitalMusicology.Corpora.Directory ~/.julia/packages/DigitalMusicology/6r3xS/src/corpora/Directory.jl:67
┌ Warning: unfolding for time signatures is not implemented yet!
└ @ DigitalMusicology.Corpora.Directory ~/.julia/packages/DigitalMusicology/6r3xS/src/corpora/Directory.jl:67
